In [19]:
from Utils.my_utils import *
from Recommenders.KNN.ItemKNNCFRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.NonPersonalizedRecommender import *
import os

In [20]:
np.random.seed(1234)
users, items, users_to_eval = readData()
URM, user_originalID_to_index, item_originalID_to_index, item_index_to_originalID = preProcess()
URM_train_full, URM_train, URM_test, URM_validation = splitURM(URM)

In [21]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2602 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2227 (17.6%) Users that have less than 1 test interactions


In [22]:
output_folder_path = "result_best_models/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [23]:
# load the model with all best parameters
recommender_object = ItemKNNCFRecommender(URM_train_full)
recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip")

ItemKNNCFRecommender: URM Detected 213 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 102 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'result_best_models/ItemKNNCFRecommender_my_own_save.zip'
ItemKNNCFRecommender: Loading complete


In [24]:
# define and load recommender
recommender_object_top_pop = TopPop(URM_train_full)
recommender_object_top_pop.load_model(output_folder_path, file_name = recommender_object_top_pop.RECOMMENDER_NAME + "_my_own_save.zip")

TopPopRecommender: URM Detected 213 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 102 ( 0.5%) items with no interactions.
TopPopRecommender: Loading model from file 'result_best_models/TopPopRecommender_my_own_save.zip'
TopPopRecommender: Loading complete


In [25]:
# recommend items to users
rec_items_ind = []
for user_id in users_to_eval:
    if(user_id in users):
        # recommend users for which we already have some ratings
        rec_items_ind.append(recommender_object.recommend(user_originalID_to_index[user_id], cutoff=10))
    else:
        # recommend users with no stored rating
        rec_items_ind.append(recommender_object_top_pop.recommend(user_id, cutoff=10, remove_seen_flag=False))

In [26]:
# evaluate recommender
evaluate_algorithm(URM_test, recommender_object)

Recommender results are: Precision = 0.1162, Recall = 0.0942, MAP = 0.0935


In [27]:
storeRecommendations(users_to_eval, rec_items_ind, item_index_to_originalID)